In [37]:
# import pandas as pd
# compare_df = pd.read_csv('smilarity_result.csv')
# compare_df[compare_df['Description choose columns'] > 145][['Question','Expect Columns','Description choose columns']]

In [2]:
from Description_base_linking import SchemaLinking

cat_cols = ['event_name', 'device_category', 'device_mobile_brand_name', 'device_mobile_model_name', 'device_mobile_marketing_name',
            'device_mobile_os_hardware_model', 'device_operating_system', 'device_operating_system_version', 'device_language', 'geo_continent',
            'geo_country', 'geo_region', 'geo_city', 'geo_sub_continent', 'geo_metro', 'app_info_version', 'app_info_install_source', 'platform',
            'firebase_previous_class', 'firebase_screen_class']
obj = SchemaLinking()
# obj.set_schema()
obj.append_description_by_category(cat_cols)

In [3]:
print(obj.schema_description_df.loc[obj.schema_description_df['Column'] == 'event_name']['Description'].tolist())

['The name of the event.settings_change_language, payatmcht_search, screen_view, pointx_mypointx_bottom_bar, pointx_ecatalog, myecoupon_valid_landing, pointx_highlight_landing, app_clear_data, pointx_point_transfer_menu, payatmcht_landing, onboard_lifestyle_landing, pointx_search, mypointx_this_month, onboard_landing, payatmcht_select_product, settings_delivery_addr_btn, virtualmall_detail_conditions_tab, settings_btn, pin_enter_landing, app_remove, mypointx_landing, onboard_highlight_landing, settings_home_landing, settings_myacc_landing, pointx_home_bottom_bar, settings_menu_landing, delivery_addr_landing, pointx_virtualmall_bottom_bar, delivery_addr_item, os_update, pointx_payatmerchant_menu, pointx_view_mypoint, onboard_permission_notification_landing, pointx_payandmerchant_bottom_bar, payatmcht_category, first_open, settings_select_lifestyle, app_update, onboard_permission_notification_allow, pointx_scanner_bottom_bar, payatmcht_confirm, scanToPay_listing_category, settings_securi

In [4]:
# add event categories to event_name description
obj.ques_col_similarity("how many user remove app","event_name")

0.09481267631053925

In [5]:
# raw event_name description ("The name of the event")
obj = SchemaLinking()
# obj.set_schema()
obj.ques_col_similarity("how many user remove app","event_name")

-0.021025139838457108

In [6]:
from Description_base_linking import SchemaLinking
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sentence_transformers import InputExample, losses
# import time

obj = SchemaLinking()
# obj.set_schema()
threshold = 0.2
compare_df = pd.read_csv('smilarity_result.csv')
questions = compare_df['Question'].to_list()
sql_queries = compare_df['SQL'].to_list()
train_ques_col = []

for i, q in enumerate(questions[:5]):
    print(q)
    expect_col = obj.columns_from_query(sql_queries[i])
    choose_col = obj.get_columns_threshold(q, threshold)

    # print("EXPECT COLUMNS:\t\t", expect_col)
    # print('SELECTED COLUMNS:\t', choose_col)

    expect_col_vec = obj.choose_col_onehot(expect_col)
    choose_col_vec = obj.choose_col_onehot(choose_col)
    accuracy = obj.accuracy_of_onehot(choose_col_vec, expect_col_vec)

    not_selected_col_idx =  (choose_col_vec == 0) & (expect_col_vec ==1 ).astype(int)
    not_selected_col = [ obj.schema_columns[idx] for idx in np.where(not_selected_col_idx == 1)[0]]
    if len(not_selected_col):
        print("COL NOT SELECTED:\t",not_selected_col)
        train_ques_col.append((q, not_selected_col))
    print(accuracy)
    print()

How many daily active users each day?
COL NOT SELECTED:	 ['event_date', 'ga_session_id']
0.5

How many monthly active users each month?
COL NOT SELECTED:	 ['user_pseudo_id', 'engagement_time_msec', 'ga_session_id']
0.25

What is the average number of daily active users last 7 days?
COL NOT SELECTED:	 ['event_date', 'user_pseudo_id', 'ga_session_id']
0.25

What is the mean number of daily active users from last 30 days?
COL NOT SELECTED:	 ['user_pseudo_id', 'ga_session_id']
0.5

When was the last time each user was active?
1.0



In [7]:
question = "How many daily active users each day?"

In [8]:
len(obj.get_columns_threshold(question))

25

In [9]:
obj.ques_col_similarity(question, 'event_date')

0.15984684228897095

In [10]:
train_ques_col[:1]

[('How many daily active users each day?', ['event_date', 'ga_session_id'])]

In [11]:
obj.fine_tune_model(train_ques_col, 100)

Fine-tune model done!


In [12]:
len(obj.get_columns_threshold(question))

26

In [13]:
obj.ques_col_similarity(question, 'event_date')

0.1683889627456665

In [14]:
obj.schema_description_df

,Table_name,Column,Description,Description_vector
0,pointx_fbs_rpt_dly,event_date,The date on which the event was logged (YYYYMM...,"[0.05822387, 0.011792056, 0.002836911, -0.0483..."
1,pointx_fbs_rpt_dly,event_month,The year month on which the event was logged (...,"[0.04541466, 0.056418423, -0.043949254, 0.0050..."
2,pointx_fbs_rpt_dly,event_bundle_sequence_id,The sequential ID of the bundle in which these...,"[-0.11162997, 0.04600868, -0.003680154, 0.0039..."
3,pointx_fbs_rpt_dly,event_timestamp,"The time (in microseconds, UTC) at which the e...","[-0.015670119, 0.066508956, -0.019871894, -0.0..."
4,pointx_fbs_rpt_dly,event_name,The name of the event.,"[-0.004712278, 0.10794155, -0.009979284, 0.052..."
...,...,...,...,...
177,pointx_fbs_rpt_dly,update_with_analytics,Update with analytics,"[-0.03796504, -0.0016496641, -0.075204395, 0.0..."
178,pointx_fbs_rpt_dly,client_code,Client code,"[-0.097830884, 0.030349072, -0.06899895, -0.08..."
179,pointx_fbs_rpt_dly,client_member_id,Client member id,"[-0.09346538, 0.04337876, -0.024237148, -0.029..."
180,pointx_fbs_rpt_dly,_dl_load_ts,วันที่โหลดข้อมูล,"[-0.041596495, -0.024681728, 0.07789318, -0.00..."


In [15]:
temp_data = {
                'Column' : ['a','b','c','d','e'],
                'Description' : ['AAA', 'BBB', 'CCC', 'DDD', 'EEE']
}
schema_temp_df = pd.DataFrame(temp_data)
schema_temp_df.to_csv('src/schema_temp.csv', index=False)

In [16]:
obj.set_schema("temp",'src/schema_temp.csv')

In [17]:
obj.schema_description_df

,Table_name,Column,Description,Description_vector
0,pointx_fbs_rpt_dly,event_date,The date on which the event was logged (YYYYMM...,"[0.05822387, 0.011792056, 0.002836911, -0.0483..."
1,pointx_fbs_rpt_dly,event_month,The year month on which the event was logged (...,"[0.04541466, 0.056418423, -0.043949254, 0.0050..."
2,pointx_fbs_rpt_dly,event_bundle_sequence_id,The sequential ID of the bundle in which these...,"[-0.11162997, 0.04600868, -0.003680154, 0.0039..."
3,pointx_fbs_rpt_dly,event_timestamp,"The time (in microseconds, UTC) at which the e...","[-0.015670119, 0.066508956, -0.019871894, -0.0..."
4,pointx_fbs_rpt_dly,event_name,The name of the event.,"[-0.004712278, 0.10794155, -0.009979284, 0.052..."
...,...,...,...,...
182,temp,a,AAA,"[-0.038816746, 0.07266189, 0.02039344, 0.01641..."
183,temp,b,BBB,"[0.012744771, -0.038333405, -0.07893924, 0.037..."
184,temp,c,CCC,"[-0.10249788, -0.059087247, -0.0851878, 0.0901..."
185,temp,d,DDD,"[-0.093442746, -0.02905863, 0.06113966, -0.092..."


In [15]:
###TEST###

In [31]:
val = ("how many event occure",'event_name')

In [32]:
obj.ques_col_similarity("how many event occure",'event_name')

0.6420384645462036

In [33]:
obj.ques_col_similarity(*val)

0.6420384645462036